In [1]:
# from spleeter.separator import Separator
import os
from googletrans import Translator
import whisper
from pytube import YouTube
from pydub import AudioSegment
import cutlet

In [3]:
katsu = cutlet.Cutlet(use_foreign_spelling=False)
katsu.romaji("写真")

'Shashin'

In [53]:
# 음원 파일 경로와 출력 디렉토리 설정
song_name = 'jjok'
input_audio_file = '../../mfvr/target/' + song_name + '.mp3'  # 분리할 음원 파일 경로
output_directory = '../../mfvr/result'  # 출력 디렉토리 경로
lyrics_directory = '../../mfvr/lyrics'  # 가사 dir
# file_path = '../../mfvr/target/_goesu.txt'  # 진짜 가사 파일 경로
# with open(file_path, 'r') as file:
#     real_lyrics = file.read()

def rgbtxt(trgb=None, brgb=None, text="Default text"):
    tt = "" if trgb==None else f"\033[38;2;{trgb[0]};{trgb[1]};{trgb[2]}m"
    bt = "" if brgb==None else f"\033[48;2;{brgb[0]};{brgb[1]};{brgb[2]}m"
    return f"{tt}{bt}{text}\033[0m"

In [13]:
# Spleeter를 사용하여 2 스템(보컬 + 반주) 분리기 초기화
# separator = Separator('spleeter:2stems')

# # 음원 분리 실행
# separator.separate_to_file(input_audio_file, output_directory)

INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
INFO:tensorflow:Calling model_fn.
INFO:tensorfl

In [62]:
def whispering():
    arg = dict(word_timestamps=True, condition_on_previous_text=False
               , language="Japanese"
               , initial_prompt="Song name is 藍二乗.")
    model = whisper.load_model("medium")
    result = model.transcribe(input_audio_file, **arg)
    return result

rdict = whispering()

c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


In [51]:
rdict['segments']

[{'id': 0,
  'seek': 0,
  'start': 8.339999999999998,
  'end': 9.06,
  'text': '【歌】',
  'tokens': [50364, 15588, 29582, 15623, 50764],
  'temperature': 0.4,
  'avg_logprob': -0.349478801091512,
  'compression_ratio': 1.3464566929133859,
  'no_speech_prob': 0.3200661540031433,
  'words': [{'word': '【',
    'start': 8.339999999999998,
    'end': 9.06,
    'probability': 0.006031804718077183},
   {'word': '歌',
    'start': 9.06,
    'end': 9.06,
    'probability': 0.049049243330955505},
   {'word': '】',
    'start': 9.06,
    'end': 9.06,
    'probability': 0.31214022636413574}]},
 {'id': 1,
  'seek': 0,
  'start': 9.06,
  'end': 11.72,
  'text': '思い出すのは君の歌',
  'tokens': [50764, 8870, 1764, 7781, 2659, 35662, 24168, 2972, 29582, 50929],
  'temperature': 0.4,
  'avg_logprob': -0.349478801091512,
  'compression_ratio': 1.3464566929133859,
  'no_speech_prob': 0.3200661540031433,
  'words': [{'word': '思',
    'start': 9.06,
    'end': 9.06,
    'probability': 0.08640564233064651},
   {'word':

In [63]:
def text_to_romaji(lyrics):
    katsu = cutlet.Cutlet(use_foreign_spelling=False); l = []
    for phrase in lyrics:
        l.append(katsu.romaji(phrase['text']))
    return l

romaji_list = text_to_romaji(rdict['segments'])
print(romaji_list)

['Kawaranai fuukei asai shoubou kouka shita ai ni jou nekorobu mama', 'Hakushi no jinsei ni hakushi no oto ga hitotsu natte iru karappo na jibun wo kyou mo utatte ita', 'Kimi ga shuyaku no plot wo egaku road no naka', 'Omottaga tsuitou seken no youth mo shasen time bottle', 'Kono jinsei sae hora pink mitai da', 'Ano koro zutto atama ni egaita yume wo mita', 'Otona ni naru hodo', 'Kodoku ni natte iku', 'Tada', 'Tada kumo wo miagete mo', 'Shikai wa kyou mo nagareru mama', 'Tooku aruita yoru ni hana wo yo gu', 'Haru to mi magaru hodo ni', 'Kimi wo tada miushinau you ni', '[uta]', 'Omoide nai you ni shita wo muita jinsei wa dou ni mo dakyou de dekiteru', 'Kokoro mo unmei mo love song mo jinsei mo shinjinai', 'Shisen furenainara zenbu ga muda', 'Wata to koko wo sayume de egaita', 'Ima ni netotte kita mama de komatte iru', 'Tadatada mabuta no uragawa tooku egaku kimi wo mita', 'Mama no outa no yasumi ni hana wo yo gu', 'Boku no me ni mata hitotsu', 'Jinsei wa Oono Kei no renzoku na n da', 'S

In [64]:
for idx, seg in enumerate(rdict['segments']):
    for word in seg['words']:
        p = int(255*float(word['probability']))
        print(rgbtxt((0,0,0),(255,p,p),word['word']), end='')
    print()
    print(romaji_list[idx])

変わらない風景 浅い消防 降下した愛 二乗 寝転ぶまま
Kawaranai fuukei asai shoubou kouka shita ai ni jou nekorobu mama
白紙の人生に 白紙の音が一つ鳴っている 空っぽな自分を今日も歌っていた
Hakushi no jinsei ni hakushi no oto ga hitotsu natte iru karappo na jibun wo kyou mo utatte ita
君が主役のプロットを描く ロードの中
Kimi ga shuyaku no plot wo egaku road no naka
思ったが追悼 世間のユースも車線タイムボトル
Omottaga tsuitou seken no youth mo shasen time bottle
この人生さえほら ピンクみたいだ
Kono jinsei sae hora pink mitai da
あの頃ずっと頭に描いた 夢を見た
Ano koro zutto atama ni egaita yume wo mita
大人になるほど
Otona ni naru hodo
孤独になっていく
Kodoku ni natte iku
ただ
Tada
ただ雲を見上げても
Tada kumo wo miagete mo
視界は今日も流れるまま
Shikai wa kyou mo nagareru mama
遠く歩いた夜に花をよぐ
Tooku aruita yoru ni hana wo yo gu
春と見曲がるほどに
Haru to mi magaru hodo ni
君をただ見失うように
Kimi wo tada miushinau you ni
【歌】
[uta]
思い出ないように 下を向いた人生はどうにも 妥協でできてる
Omoide nai you ni shita wo muita jinsei wa dou ni mo dakyou de dekiteru
心も運命も ラブソングも人生も信じない
Kokoro mo unmei mo love song mo jinsei mo shinjinai
視線触れないなら全部が無駄
Shisen furenainara zenbu ga muda
わたとここをさ 夢で描いた
Wata to 

In [9]:
def downloadYouTube(videourl, folderpath):

    yt = YouTube(videourl)
    aud = yt.streams.filter(only_audio=True).first()
    downloaded = aud.download(filename='temp_audio.mp3')
    audio = AudioSegment.from_file(downloaded)
    audio.export(f"{folderpath}/{yt.title}.mp3", format='mp3')
    os.remove(downloaded)

video_link = 'https://www.youtube.com/watch?v=4MoRLTAJY_0'
downloadYouTube(video_link, '../../mfvr/target')